# Classes and variables

In [31]:
from birdclassification.preprocessing.filtering import filter_recordings_30
from torch.utils.data import DataLoader
import torch
from sklearn.model_selection import train_test_split
from training.dataset import Recordings30
from birdclassification.visualization.plots import plot_torch_spectrogram
from training.cnn_training_torch.CNN_model import CNNNetwork
from torchsummary import summary
from sklearn.utils import class_weight
import numpy as np

In [3]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

SEED = 123
RECORDINGS_DIR = '/mnt/d/recordings_30/'
NOISES_DIR = '/aaa/'

SAMPLE_RATE = 32000
NUM_SAMPLES = SAMPLE_RATE * 1
BATCH_SIZE = 32
NUM_WORKERS = 8

LEARNING_RATE = 0.0001
EPOCHS = 5

/home/jacek/PycharmProjects/Bird-classification-model/venv/lib/python3.11/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


# Prepare dataset and dataloaders, visualize dataset

In [4]:
df = filter_recordings_30("../../data/xeno_canto_recordings.csv", "../../data/bird-list-extended.csv", )

#subset for test purpose
# df = df.sample(frac = 0.1, random_state=SEED)

train_df, test_val_df = train_test_split(df, stratify=df['Latin name'], test_size=0.2, random_state = SEED)
val_df, test_df = train_test_split(test_val_df, stratify=test_val_df['Latin name'], test_size=0.5, random_state = SEED)

train_ds = Recordings30(train_df, recording_dir=RECORDINGS_DIR, noises_dir=NOISES_DIR, sample_rate=SAMPLE_RATE, device = DEVICE)
val_ds = Recordings30(val_df, recording_dir=RECORDINGS_DIR, noises_dir=NOISES_DIR, sample_rate = 32000, device = DEVICE)
test_ds = Recordings30(test_df, recording_dir=RECORDINGS_DIR, noises_dir=NOISES_DIR,sample_rate = 32000,device = DEVICE)

train_dl  = DataLoader(train_ds, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
val_dl  = DataLoader(val_ds, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
test_dl  = DataLoader(test_ds, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)

/home/jacek/PycharmProjects/Bird-classification-model/birdclassification/preprocessing/filtering.py:72: DtypeWarning: Columns (10,39,43,44,45,46,47,48) have mixed types. Specify dtype option on import or set low_memory=False.
  recordings = pd.read_csv(filepath_recordings)


In [39]:
class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                  classes = np.sort(df['Latin name'].unique()),
                                                  y = df.loc[:, 'Latin name']
                                                 )
class_weights

array([ 0.51893968,  1.27070614,  1.94016008,  1.08587615,  1.45975912,
        0.66487577, 10.02652068,  6.30107034,  0.66167309,  1.13336084,
        1.94290429,  0.70732921,  2.63653231,  0.55975278,  0.73065603,
        1.44440939,  0.82500501,  1.4706995 ,  0.55188161,  6.54111111,
        2.13961578,  1.02971014,  1.20494152,  1.33752029,  3.8585206 ,
        0.8065962 ,  9.15755556,  0.20450102,  1.63722686,  1.14756335])

In [12]:
train_ds.get_mapping()

{0: 'Alauda arvensis',
 1: 'Anas platyrhynchos',
 2: 'Apus apus',
 3: 'Asio otus',
 4: 'Buteo buteo',
 5: 'Carduelis carduelis',
 6: 'Ciconia ciconia',
 7: 'Columba livia',
 8: 'Corvus corax',
 9: 'Corvus cornix',
 10: 'Corvus frugilegus',
 11: 'Cuculus canorus',
 12: 'Cygnus olor',
 13: 'Dendrocopos major',
 14: 'Garrulus glandarius',
 15: 'Grus grus',
 16: 'Hirundo rustica',
 17: 'Lophophanes cristatus',
 18: 'Passer domesticus',
 19: 'Phalacrocorax carbo',
 20: 'Phasianus colchicus',
 21: 'Phoenicurus ochruros',
 22: 'Pica pica',
 23: 'Picus viridis',
 24: 'Sternula albifrons',
 25: 'Sturnus vulgaris',
 26: 'Tetrao urogallus',
 27: 'Turdus merula',
 28: 'Turdus torquatus',
 29: 'Turdus viscivorus'}

In [ ]:
train_ds.visualize_dataset(3207,5)

# Prepare a model, loss functions

In [ ]:
cnn = CNNNetwork().to(DEVICE)
summary(cnn, (1, 64, 251)) 

In [ ]:
cnn.eval()

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(),
                             lr=LEARNING_RATE)

# Train loop

In [21]:
from datetime import datetime
import sys
from training.training_utils import train_one_epoch
from torch.utils.tensorboard import SummaryWriter
from training.validation_metrics import calculate_metric
from sklearn.metrics import f1_score

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter(f'logs/fashion_trainer_{timestamp}')
epoch_number = 0

best_vloss = sys.float_info.max

for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))
    
    # Make sure gradient tracking is on, and do a pass over the data
    cnn.train(True)
    avg_loss = train_one_epoch(epoch_number, writer, train_dl, optimizer, loss_fn, cnn, DEVICE)
    
    # Set the model to evaluation mode, disabling dropout and using population 
    # statistics for batch normalization.
    cnn.eval()
    running_vloss = 0.0

    # Disable gradient computation and reduce memory consumption.
    with torch.no_grad():
        for i, vdata in enumerate(val_dl):
            vinputs, vlabels = vdata
            vinputs = torch.unsqueeze(vinputs, dim=1)
            voutputs = cnn(vinputs)
            vloss = loss_fn(voutputs, vlabels)
            running_vloss += vloss
    
    avg_vloss = running_vloss / (i + 1)
    print("#############################################################")
    print("Epoch results:")
    print(f'Loss train {avg_loss} valid loss: {avg_vloss}')
    validation_f1_score = calculate_metric(cnn, val_dl, metric=f1_score)
    train_f1_score = None
    print(f'F1 score train {train_f1_score} valid f1 score {validation_f1_score}')
    print("#############################################################\n\n")
    
    # Log the running loss averaged per batch
    # for both training and validation
    writer.add_scalars('Training vs. Validation Loss',
                    { 'Training' : avg_loss, 'Validation' : avg_vloss },
                    epoch_number + 1)
    
    
    writer.add_scalars('Macro_averaged_f1_score',
                    { 'Validation' : validation_f1_score},
                    epoch_number + 1)
    
    writer.flush()
    
    # Track best performance, and save the model's state
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path = f'model_{timestamp}_{epoch_number}'
        torch.save(cnn.state_dict(), model_path)
    
    epoch_number += 1

2023-11-28 20:02:44.270796: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-28 20:02:44.414751: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-28 20:02:44.816148: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-28 20:02:45.910458: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


EPOCH 1:


NameError: name 'cnn' is not defined

# Save the model

In [ ]:
torch.save(cnn.state_dict(),"saved_models/cnn_1.pt")